In [ ]:
#@title Install Dependencies

!pip install sentence-transformers
!pip install torch

In [ ]:
#@title Load Embedding Model
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("Snowflake/snowflake-arctic-embed-m")

In [ ]:
#@title Upload Intents and Examples CSV
from google.colab import files
import csv

uploaded = files.upload()

for filename in uploaded.keys():
  print(f"Uploaded: {filename}")

def read_csv(filename):
  intents = []
  examples = []

  with open(filename, "r", newline="") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      intents.append(row["intent"])
      examples.append(row["example"])

  return intents, examples

intent_values, example_values = read_csv("output.csv")
print(example_values)

In [8]:
#@title Generate Embeddings from Examples

examples_embeddings = model.encode(example_values, convert_to_tensor=True)

In [14]:
#@title Determine Intent from Query

query = "I want to enable 2 factro for my account" # @param {type:"string"}

top_k = min(5, len(examples_embeddings))

query_embedding = model.encode(query, convert_to_tensor=True)

hits = util.semantic_search(query_embedding, examples_embeddings, top_k=5)
hits = hits[0]

print("Query:", query)
print("Top 5 similar examples:")

for hit in hits:
    print(intent_values[hit['corpus_id']], example_values[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))


Query: I want to enable 2 factro for my account
Top 5 similar examples:
intent_enable_2fa Enable 2FA (Score: 0.8144)
intent_core_help I need to deploy (Score: 0.8122)
intent_enable_2fa turn on 2fa (Score: 0.8118)
intent_access_request Request access from my administrator (Score: 0.8088)
intent_integration_setup can you help me setup integrations (Score: 0.8088)
